<a href="https://colab.research.google.com/github/KimDaeUng/t2i_bert_finetuning/blob/master/BERT_fine_tuning_on_CUB_Captions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fine-tuning BERT on CUB Captions (domain adaptation)

In [ ]:
!pip install nlp
!pip install transformers

In [ ]:
import os
from glob import glob

## 1. Prepare dataset

In [ ]:
# Mount google drive
from os import path
from google.colab import drive

drive.mount('/content/drive')

In [ ]:
# Download the caption file
!wget "https://github.com/KimDaeUng/t2i_bert_finetuning/blob/master/text_c10.zip" -O '/content/drive/My Drive/Colab Notebooks/t2i/data/text_c10.zip'

In [ ]:
# Unzip the caption file
!unzip '/content/drive/My Drive/Colab Notebooks/t2i/data/text_c10.zip' -d '/content/drive/My Drive/Colab Notebooks/t2i/data/text_c10'

In [ ]:
# Get text file path list
root_text_path = "/content/drive/My Drive/Colab Notebooks/t2i/data/text_c10/*/*.txt"
text_list = glob(root_text_path)

In [ ]:
# Check the path list
text_list[:5]

In [22]:
from nlp import load_dataset

Using custom data configuration default


Dataset text downloaded and prepared to /root/.cache/huggingface/datasets/text/default-07f6fa169a8f98f9/0.0.0/3a79870d85f1982d6a2af884fde86a71c771747b4b161fd302d28ad22adf985b. Subsequent calls will reuse this data.


In [36]:
dataset = load_dataset('text', data_files=text_list)

Using custom data configuration default


Dataset text downloaded and prepared to /root/.cache/huggingface/datasets/text/default-6f7df17d102c4e7c/0.0.0/3a79870d85f1982d6a2af884fde86a71c771747b4b161fd302d28ad22adf985b. Subsequent calls will reuse this data.


In [53]:
# Check the path list
dataset['train']['text'][:5]

['the medium sized bird has a dark grey color, a black downward curved beak, and long wings.\n',
 'the bird is dark grey brown with a thick curved bill and a flat shaped tail.\n',
 'bird has brown body feathers, white breast feathers and black beak\n',
 'this bird has a dark brown overall body color, with a small white patch around the base of the bill.\n',
 'the bird has very long and large brown wings, as well as a black body and a long black beak.\n']

In [56]:
# Save dataset as single text file
path_lm_training = "/content/drive/My Drive/Colab Notebooks/t2i/data/text_c10/whole_captions.txt"
with open(path_lm_training, 'w') as f:
    for item in dataset['train']['text']:
        f.write(item)

100%|██████████| 117880/117880 [00:00<00:00, 1225145.34it/s]


In [70]:
# Set BERT tokenizer
from transformers import BertTokenizer
path_model = "/content/drive/My Drive/Colab Notebooks/t2i/BERT_CUB"
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True, cache_dir=path_model)

In [58]:
# Make encoded dataset 
%%time
from transformers import LineByLineTextDataset

dataset = LineByLineTextDataset(
    tokenizer=tokenizer,
    file_path=path_lm_training,
    block_size=128,
)

CPU times: user 33.2 s, sys: 6.07 ms, total: 33.3 s
Wall time: 33.4 s


## 2. Train a language model from pretrained weights(domain adaptation)

In [49]:
from transformers import BertForMaskedLM
model = BertForMaskedLM.from_pretrained('bert-base-uncased')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForMaskedLM were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['cls.predictions.decoder.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [44]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.15
)

In [59]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir=path_model,
    overwrite_output_dir=True,
    num_train_epochs=1,
    per_gpu_train_batch_size=64,
    save_steps=10_000,
    save_total_limit=2,
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset,
    prediction_loss_only=True,
)

/usr/local/lib/python3.6/dist-packages/transformers/trainer.py:247: FutureWarning: Passing `prediction_loss_only` as a keyword argument is deprecated and won't be possible in a future version. Use `args.prediction_loss_only` instead.
  FutureWarning,


In [60]:
%%time
trainer.train()

Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.
Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.
Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.


{'loss': 1.2871446533203126, 'learning_rate': 3.642779587404995e-05, 'epoch': 0.2714440825190011, 'step': 500}
{'loss': 1.1318653564453125, 'learning_rate': 2.2855591748099893e-05, 'epoch': 0.5428881650380022, 'step': 1000}
{'loss': 1.080734375, 'learning_rate': 9.283387622149838e-06, 'epoch': 0.8143322475570033, 'step': 1500}


CPU times: user 7min 6s, sys: 3min 10s, total: 10min 16s
Wall time: 10min 21s


TrainOutput(global_step=1842, training_loss=1.1457514229607764)

In [62]:
trainer.save_model(path_model)

## 3. Check that LM actually trained

In [74]:
tokenizer.special_tokens_map

{'cls_token': '[CLS]',
 'mask_token': '[MASK]',
 'pad_token': '[PAD]',
 'sep_token': '[SEP]',
 'unk_token': '[UNK]'}

In [72]:
from transformers import pipeline

fill_mask = pipeline(
    "fill-mask",
    model=path_model,
    tokenizer=tokenizer
)

In [78]:
fill_mask("The bird has head with [MASK].")

[{'score': 0.26184365153312683,
  'sequence': '[CLS] the bird has head with black. [SEP]',
  'token': 2304,
  'token_str': 'black'},
 {'score': 0.1308348923921585,
  'sequence': '[CLS] the bird has head with white. [SEP]',
  'token': 2317,
  'token_str': 'white'},
 {'score': 0.058785539120435715,
  'sequence': '[CLS] the bird has head with red. [SEP]',
  'token': 2417,
  'token_str': 'red'},
 {'score': 0.05665842443704605,
  'sequence': '[CLS] the bird has head with brown. [SEP]',
  'token': 2829,
  'token_str': 'brown'},
 {'score': 0.05123694986104965,
  'sequence': '[CLS] the bird has head with grey. [SEP]',
  'token': 4462,
  'token_str': 'grey'}]

## References
- [huggingface examples - language modeling](https://colab.research.google.com/github/huggingface/blog/blob/master/notebooks/01_how_to_train.ipynb#scrollTo=VmaHZXzmkNtJ)  
- [BERT Fine-Tuning Tutorial with PyTorch, Chris McCormick and Nick Ryan](https://mccormickml.com/2019/07/22/BERT-fine-tuning/#31-bert-tokenizer)